# Import libraries

In [1]:
import os, glob, csv, sys, re
from os import path
import pandas as pd
from bs4 import BeautifulSoup

# Define directories

In [2]:
number = '16'
userhome = os.path.expanduser('~')
dataset = userhome + r'/mining-forum/data/forumfiles_' + number + '/'
analyze_dir = userhome + r'/mining-forum/analyze/'

# Counting number of files

In [3]:
file_list = glob.iglob(dataset + '*')
filenum = ';'.join(file_list).split(';')
filenum.sort()
print ("Found " + str(len(filenum)) + " files")

Found 44000 files


# Clean the noisy data (remove duplicate files)

In [4]:
file = []
mid_list = []
num = 0
for files in glob.iglob(dataset + "*"):    
    num += 1
    sys.stdout.write('\rExtracting file number: %i' %(num) + ' / %i' %(len(filenum)) + ' --> %s' %(files))
    sys.stdout.flush()

    f = open(files)
    f = f.read()
    
    file_num = re.search('\/(\d+)', files)
    file_num = file_num.group(1)
    
    try:
        soup = BeautifulSoup(f, 'html.parser')
        topic = soup.find('a', attrs={'class':'MsgSubText'})
        for title in topic.stripped_strings:
            title = title

        msid = []
        soup = BeautifulSoup(f, 'html.parser')
        for message_id in soup.find_all('td', attrs={'class':'MsgR1 vt al MsgSubText'}):
            for mid in message_id.find_all('span', attrs={'class':'SmallText'}):
                for msgid in mid.find_all('a'):
                    msid.append(msgid.text)
        msid = msid[0]

        data = [file_num, msid, title]
        username = []
        uname = []
        href = []
        soup = BeautifulSoup(f, 'html.parser')
        for userlink in soup.find_all('td', attrs={'class':'MsgSpacer'}):
            for user in userlink.find_all('td', attrs={'class':'msgud'}):
                for usr in user.find_all('a', href=True):
                    href.append(usr['href'])                                        
                    uname.append(usr.text)
                    if uname[0] == '':
                        uname[0] = 'Eclipse User'
                    username = [uname[0],href[0]]
                    
                    try:
                        aut_id = re.search('(\d+)', username[1])
                        aut_id = aut_id.group()
                    except:
                        aut_id = '0'
                    
                    username.insert(len(username),aut_id)

        if msid not in mid_list:
            mid_list.append(msid)
            clean_file = [file_num, msid, title, username[0], username[1], username[2]]
            file.append(clean_file)
        else:
            pass

    except:
        pass
    
print ("\nFiltering files are finished")

Extracting file number: 44000 / 44000 --> /Users/YusufNugroho/mining-forum/data/forumfiles_16/191655
Filtering files are finished


In [5]:
file.sort()
file

[['176000',
  'message #560566',
  'Announcing EclipsePro',
  'Mark Johnson',
  'index.php?t=usrinfo&id=23029&',
  '23029'],
 ['176001',
  'message #560570',
  'Eclipse 3.6',
  'Eclipse',
  'index.php?t=usrinfo&id=72112&',
  '72112'],
 ['176002',
  'message #560571',
  'test',
  'Ger Kehoe',
  'index.php?t=usrinfo&id=44400&',
  '44400'],
 ['176003',
  'message #560572',
  '[oaw] Problem invoking JavaMetamodel from plugin',
  'Eclipse User',
  '/donate',
  '0'],
 ['176004',
  'message #560574',
  'help for making a modelling editor in PDE',
  'neha',
  'index.php?t=usrinfo&id=43288&',
  '43288'],
 ['176005',
  'message #560576',
  'repost from stem-dev@eclipse.org',
  'Daniel Ford',
  'index.php?t=usrinfo&id=27941&',
  '27941'],
 ['176006',
  'message #560577',
  'Awoma releases ThinStructure Framework 3.1.1',
  'Eclipse User',
  '/donate',
  '0'],
 ['176007', 'message #560580', 'test666', 'Eclipse User', '/donate', '0'],
 ['176008',
  'message #560585',
  'Welcome to eclipse.pave',
  '

In [6]:
len(file)

35306

In [7]:
with open (analyze_dir + 'csv_files/clean_files/with_author_id/clean_files_forum_with_author_id_' + number + '.csv', 'w') as csvfile:
    writers = csv.writer(csvfile, delimiter = ",")
    header = ['file_number', 'message_id', 'title', 'author', 'author_link', 'author_id']
    writers.writerow(header)
    for item in file:
        writers.writerow(item)
print ('CSV file has been created')

CSV file has been created


In [8]:
df_thread = pd.read_csv(analyze_dir + 'csv_files/clean_files/with_author_id/clean_files_forum_with_author_id_' + number + '.csv')
df_thread

,file_number,message_id,title,author,author_link,author_id
0,176000,message #560566,Announcing EclipsePro,Mark Johnson,index.php?t=usrinfo&id=23029&,23029
1,176001,message #560570,Eclipse 3.6,Eclipse,index.php?t=usrinfo&id=72112&,72112
2,176002,message #560571,test,Ger Kehoe,index.php?t=usrinfo&id=44400&,44400
3,176003,message #560572,[oaw] Problem invoking JavaMetamodel from plugin,Eclipse User,/donate,0
4,176004,message #560574,help for making a modelling editor in PDE,neha,index.php?t=usrinfo&id=43288&,43288
5,176005,message #560576,repost from stem-dev@eclipse.org,Daniel Ford,index.php?t=usrinfo&id=27941&,27941
6,176006,message #560577,Awoma releases ThinStructure Framework 3.1.1,Eclipse User,/donate,0
7,176007,message #560580,test666,Eclipse User,/donate,0
8,176008,message #560585,Welcome to eclipse.pave,Eclipse Webmaster,index.php?t=usrinfo&id=9957&,9957
9,176009,message #560588,test,Eclipse User,/donate,0


# Remove threads if the author is Eclipse Webmaster

In [9]:
cols = ['file_number', 'message_id', 'title', 'author', 'author_link', 'author_id']
new_df_thread = df_thread[cols][df_thread[cols]['author_id'] != 9957]
new_df_thread

,file_number,message_id,title,author,author_link,author_id
0,176000,message #560566,Announcing EclipsePro,Mark Johnson,index.php?t=usrinfo&id=23029&,23029
1,176001,message #560570,Eclipse 3.6,Eclipse,index.php?t=usrinfo&id=72112&,72112
2,176002,message #560571,test,Ger Kehoe,index.php?t=usrinfo&id=44400&,44400
3,176003,message #560572,[oaw] Problem invoking JavaMetamodel from plugin,Eclipse User,/donate,0
4,176004,message #560574,help for making a modelling editor in PDE,neha,index.php?t=usrinfo&id=43288&,43288
5,176005,message #560576,repost from stem-dev@eclipse.org,Daniel Ford,index.php?t=usrinfo&id=27941&,27941
6,176006,message #560577,Awoma releases ThinStructure Framework 3.1.1,Eclipse User,/donate,0
7,176007,message #560580,test666,Eclipse User,/donate,0
9,176009,message #560588,test,Eclipse User,/donate,0
10,176010,message #560589,New Eclipse Resource Catalog,Ian Skerrett,index.php?t=usrinfo&id=22793&,22793


In [10]:
new_df_thread.to_csv(analyze_dir + 'csv_files/clean_files/with_author_id/cleaned/threads_without_bot_authors_forum_' + number + '.csv', index=False)
print ("CSV file has been created")

CSV file has been created
